In [1]:
import os
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [2]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [3]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [4]:
print(fountains.crs)

None


In [5]:
fountains = fountains.set_crs('epsg:4326')

In [6]:
print(fountains.crs)

epsg:4326


In [7]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [8]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Express bike path (REV)

In [9]:
WGS84=4326
MTM8=32188

rev_url="https://donnees.montreal.ca/dataset/8a4bf03c-dff6-4add-b58b-c38954b0ed0d/resource/8ad67029-cf2e-49ae-a4b6-20d31611ab6e/download/reseau-express-velo.geojson"

def process_request(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()


def json_items_to_geodataframe(http_response):
    return gpd.GeoDataFrame().from_features(http_response).set_crs(f"epsg:{WGS84}")


http_response = process_request(rev_url)
rev = json_items_to_geodataframe(http_response)
rev.columns = rev.columns.str.lower()

In [10]:
print(rev.crs)

epsg:4326


In [11]:
columns = ["id", "nom_arr_vi", "geometry"]
rev = rev.filter(columns)

### calcul the colsest distance between fountain and REV

In [12]:
fountains_utm = fountains.to_crs(epsg=32188)
rev_utm = rev.to_crs(epsg=32188)

In [13]:
print(fountains_utm.crs)
print(rev_utm.crs)

EPSG:32188
EPSG:32188


In [14]:
fountain_w_rev_data = fountains_utm.sjoin_nearest(rev_utm, distance_col="nearest_rev")

In [15]:
print(f"min: {min(fountain_w_rev_data['nearest_rev'])}")
print(f"max: {max(fountain_w_rev_data['nearest_rev'])}")

min: 10.765171370946362
max: 23190.890597142912


In [16]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [17]:
part = fountain_w_rev_data[fountain_w_rev_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id_left"]).add_to(m)

In [18]:
m

In [19]:
part = rev.copy()
folium.Choropleth(part.geometry, line_weight=2, line_color="blue").add_to(m)

In [20]:
m